<a href="https://colab.research.google.com/github/bbeck0744/HeliuMN/blob/main/HeliuMN_Hotspot_Miner_Summaries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Load Data From Helium Blockchain**
---

In [ ]:
#@title Press the play button to the left of this text to load data from each site. Once the button stops moving you should be able to click on the subsequent play buttons below to update the graphs.
 
import pandas as pd
from functools import partial, reduce
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import json
import plotly.express as px
import plotly.graph_objects as go
import time

dataframe_daily = pd.DataFrame()

retry_strategy = Retry(
    total=100,
    status_forcelist=[429, 500, 502, 503, 504],
    method_whitelist=["HEAD", "GET", "OPTIONS"]
)
adapter = HTTPAdapter(max_retries=retry_strategy)
http = requests.Session()
http.mount("https://", adapter)
http.mount("http://", adapter)
##########################################
df_hourly = {}

addresses = ['11oqkFjrVDA34MNHpUB2Usr1mpAhytYZbEQNRrzeqKRzabxEFA7', '11MDEy4qcRkLXtwMqK4CGp6a3vX5eTVa44YcN65JHKSLYvw2ZJV']
for address in addresses:
    response = http.get(f'https://api.helium.io/v1/hotspots/{address}/rewards/sum/?min_time=-24%20hour&bucket=hour')
    responsedict = json.loads(response.text)
    timestamp_list = []
    hnt_list = []
    for time in range(0, len(responsedict['data'])):
        timestamp_list.append(responsedict['data'][time]['timestamp'])
        hnt_list.append(responsedict['data'][time]['max'])
    dict = {'date_time': timestamp_list, f'{address}_hnt': hnt_list}
    #df[count] = pd.DataFrame(dict)
    df_hourly[address] = pd.DataFrame(dict)

my_reduce_hourly = partial(pd.merge, on='date_time', how='outer')
dataframe_hourly = reduce(my_reduce_hourly, df_hourly.values()) 

############################################
df_daily = {}

for address in addresses:
    response = http.get(f'https://api.helium.io/v1/hotspots/{address}/rewards/sum/?min_time=-30%20day&bucket=day')
    responsedict = json.loads(response.text)
    timestamp_list = []
    hnt_list = []
    for time in range(0, len(responsedict['data'])):
        timestamp_list.append(responsedict['data'][time]['timestamp'])
        hnt_list.append(responsedict['data'][time]['total'])
    dict = {'date_time': timestamp_list, f'{address}_hnt': hnt_list}
    #df[count] = pd.DataFrame(dict)
    df_daily[address] = pd.DataFrame(dict)
    
my_reduce_daily = partial(pd.merge, on='date_time', how='outer')
dataframe_daily = reduce(my_reduce_daily, df_daily.values()) 
###########################################
dataframe_daily['daily_sum'] = dataframe_daily['11oqkFjrVDA34MNHpUB2Usr1mpAhytYZbEQNRrzeqKRzabxEFA7_hnt'] + dataframe_daily['11MDEy4qcRkLXtwMqK4CGp6a3vX5eTVa44YcN65JHKSLYvw2ZJV_hnt']
dataframe_daily = dataframe_daily.sort_values(by='date_time')
dataframe_daily = dataframe_daily.reset_index()
dataframe_daily['Cumulative_sum'] = dataframe_daily['daily_sum'].cumsum()
dataframe_daily['date_time'] = pd.to_datetime(dataframe_daily['date_time'], utc=True)


## **Hourly HNT Rewards by Hotspot**
---

In [ ]:
#@title This figure represents hourly HNT rewards for each hotspot over the last 24 hours
fig = go.Figure()
fig.add_trace(go.Bar(
    x=dataframe_hourly['date_time'],
    y=dataframe_hourly['11oqkFjrVDA34MNHpUB2Usr1mpAhytYZbEQNRrzeqKRzabxEFA7_hnt'],
    name='Clean Linen Opossum (Spinner Miner)',
    marker_color='#0C7489'
))
fig.add_trace(go.Bar(
    x=dataframe_hourly['date_time'],
    y=dataframe_hourly['11MDEy4qcRkLXtwMqK4CGp6a3vX5eTVa44YcN65JHKSLYvw2ZJV_hnt'],
    name='Fun Aquamarine Owl (Beck Parents)',
    marker_color='#DAA49A'
))
fig.update_yaxes(title_text="Hourly Site HNT")
# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(barmode='group', xaxis_tickangle=-45)
fig.show()

## **Daily HNT Rewards by Hotspot**
---

In [ ]:
#@title This figure represents daily HNT rewards for each hotspot over the last 30 days
fig = go.Figure()
fig.add_trace(go.Bar(
    x=dataframe_daily['date_time'],
    y=dataframe_daily['11oqkFjrVDA34MNHpUB2Usr1mpAhytYZbEQNRrzeqKRzabxEFA7_hnt'],
    name='Clean Linen Opossum (Spinner Miner)',
    marker_color='#0C7489'
))
fig.add_trace(go.Bar(
    x=dataframe_daily['date_time'],
    y=dataframe_daily['11MDEy4qcRkLXtwMqK4CGp6a3vX5eTVa44YcN65JHKSLYvw2ZJV_hnt'],
    name='Fun Aquamarine Owl (Beck Parents)',
    marker_color='#DAA49A'
))
fig.update_yaxes(title_text="Daily Site HNT")
# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(barmode='group', xaxis_tickangle=-45)
fig.show()

## **Cumulative HNT for all HeliuMN Hotspots**
---

In [ ]:
#@title This figure represents the cumulative amount of HNT for all of our hotspots over time 
fig = go.Figure()
fig.add_trace(go.Bar(x=dataframe_daily['date_time'],y=dataframe_daily['Cumulative_sum'], marker_color='#EF8354'))
fig.update_yaxes(title_text="Cumulative HNT")

fig.show()